In [1]:
from src.utils import initialize_service_object
from src.body import BodyForGaReports , BodyForGaUserActivity
from src.service import ServiceObjectForGa

import pandas as pd
import time

# Initialize Service Object

In [2]:
service = initialize_service_object(*ServiceObjectForGa.get_params())

# Get Google Analytics Reports

In [3]:
start_date = '2020-07-31'
end_date = '2020-07-31'
m_list = ['users', 'sessions'] # metrics
d_list = ['clientId'] # dimensions
# ['ga:userType', 'ga:country', 'ga:region', 'ga:metro', 'ga:city', 'ga:browser', 'ga:deviceCategory', 'ga:operatingSystem']
# ['previousPagePath', 'pageTitle', 'landingPagePath', 'exitPagePath', 'pagePath', 'language', 'date']
# d_list = ['userGender', 'userAgeBracket']

In [4]:
ga = BodyForGaReports(start_date, end_date)
body = ga.get_body(m_list, d_list)
reports = service.reports().batchGet(body=body).execute()

In [5]:
df = ga.report_rows_to_df(reports['reports'][0]['data']['rows'])
df.head()


/Users/s.imazeki/Documents/GA/src/body/body_for_ga_reports.py:31: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = pd.io.json.json_normalize(rows)


,clientId,users,sessions
0,1002006134.1596178612,1,1
1,1006063715.1594347551,1,1
2,1011196529.1596175563,1,1
3,1011246136.1594876125,1,1
4,1013231651.1596153032,1,1


In [6]:
ids = list(df.clientId.unique())
print(len(ids))

888


# Get User Activity

In [7]:
# https://developers.google.com/analytics/devguides/config/mgmt/v3/limits-quotas?hl=ja
# https://console.developers.google.com/?hl=ja
# から ReportingAPIを選択して、上限を１０００件にあげられる
# 対象のプロジェクトに入っている状態

In [8]:
%%time
dfs = []
cnt = 1
for id in ids:
    ua = BodyForGaUserActivity(start_date, end_date)
    body = ua.get_body(id)
    activity = service.userActivity().search(body=body).execute()
    df = ua.sessions_to_df(activity['sessions'])
    df['clientId'] = id
    dfs.append(df)
    cnt += 1
    if cnt % 1000 ==0:
        time.sleep(100) # 100秒あたり100件のリクエスト制限

CPU times: user 8.66 s, sys: 219 ms, total: 8.88 s
Wall time: 13min 34s


In [12]:
df = pd.concat(dfs)
df.head()

,activityTime,source,medium,channelGrouping,campaign,keyword,hostname,landingPagePath,activityType,pagePath,...,platform,dataSource,sessionDate,clientId,goals,eventCategory,eventAction,eventLabel,eventValue,eventCount
0,2020-07-31T15:56:52.817110Z,bing,organic,Organic Search,(not set),(not provided),datumstudio.jp,/blog/データ分析を強みとする上場企業各社の9月期決算,PAGEVIEW,/blog/データ分析を強みとする上場企業各社の9月期決算,...,Windows,web,2020-07-31,1002006134.1596178612,NaN,NaN,NaN,NaN,NaN,NaN
0,2020-07-31T10:22:17.604371Z,google,organic,Organic Search,(not set),(not provided),datumstudio.jp,/information/0519game_ms,PAGEVIEW,/information/0519game_ms,...,Macintosh,web,2020-07-31,1006063715.1594347551,NaN,NaN,NaN,NaN,NaN,NaN
0,2020-07-31T15:10:10.781258Z,google,organic,Organic Search,(not set),(not provided),datumstudio.jp,/blog/連載：azure-で学ぶインフラ入門３（dbサーバー構築編,PAGEVIEW,/blog/datum-studio社員旅行2018in韓国ソウル,...,Windows,web,2020-07-31,1011196529.1596175563,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-07-31T15:09:58.976616Z,google,organic,Organic Search,(not set),(not provided),datumstudio.jp,/blog/連載：azure-で学ぶインフラ入門３（dbサーバー構築編,PAGEVIEW,/blog/【r】主成分分析と探索的因子分析で顧客満足度を,...,Windows,web,2020-07-31,1011196529.1596175563,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-07-31T15:09:45.966822Z,google,organic,Organic Search,(not set),(not provided),datumstudio.jp,/blog/連載：azure-で学ぶインフラ入門３（dbサーバー構築編,PAGEVIEW,/blog/pandasでcsvファイルを読み書き,...,Windows,web,2020-07-31,1011196529.1596175563,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.to_csv('./user_activity.csv', index=False, header=True)